# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'Data'

import pandas as pd
import numpy as np

import os
import glob
import re

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Outline / Methodology

#### Data organisation
The data for this task is organised in three directories, one for each country. Within each directory there are the virus data for the corresponding country, split to several CSV files, one for each day of a month (all data files concern year 2014).

By looking at the data files, we observe that the periods they cover are not the same. For example, the data for Guinea cover most days of September and just a few days of August, the data for Liberia cover the period from June to December, while the data for Sierra Leone cover the period from August to December. Futhermore, the periods are not continuous, i.e. there are days for which data files are missing. These facts should not affect our analysis.

#### Methodology
For reasons that will become clear later, but that are mostly attributed to missing data, we will aggregate total (suspected+probable+confirmed) cases and deaths. The idea behind this decision is that in cases that data is missing for one of the three categories, we just aggregate the data available.

The first step for each country is to normalise their datasets, namely bring them to a common format, which we will be able to process later in a general way. This normalisation runs some checks on the data and tries to infer and fill in some missing data. For example, some Liberia data files contain the individual numbers for suspected, probable and confirmed cases or deaths, but not the total. In places where it can be safely infered, we fill this value in.

Furthermore, we make checks to find out which categories it is safe to use for the data aggregation. For example, if we do not have enough data for a category, we do not use it (in the sense that we will pick some other variable for the final `DataFrame`). If a category makes no sense (e.g. the end-start of month difference of cumulative sums is negative - that is we resurrected people), we don't use it either.

#### Daily vs. Cumulative data
When considering the required data analysis, we can point out two possible ways of carrying it out:
1. Calculate the mean of daily cases/deaths. This approach will work well if the data indicating daily cases/deaths is consistent and correct. If many such values are missing or, even worse, are falselly zero, the average will suffer. As we will see, the daily data for all three countries is generally either missing, or inconsistent.
2. Aggregate as much cumulative data as possible for the available periods. Then, calculate the difference of the cumulative values for a given month and divide by the number of days available for that month. This a way safer way of calculating things, as we do not depend of each individual day data. If, say, a day's data slipped, this will not affect the overall calculation. Or if, say, a correction was made, we don't need to bother with it, as it will be reflected to the rest of the data.

Our choice was to go with the second approach. The reason will become more clear, as soon as we present the analysis of the data for each country.

We begin by importing some modules and defining some constants...

In [ ]:
from datetime import datetime

columns_to_check_cases = [
    'ADA_NEW_CASES_SUSPECT',
    'ADA_NEW_CASES_PROBABLE',
    'ADA_NEW_CASES_CONFIRMED',
    'ADA_NEW_CASES_TOTAL',
    'ADA_CUM_CASES_SUSPECT',
    'ADA_CUM_CASES_PROBABLE',
    'ADA_CUM_CASES_CONFIRMED',
    'ADA_CUM_CASES_TOTAL',
]

columns_to_check_deaths = [
    'ADA_NEW_DEATHS_SUSPECT',
    'ADA_NEW_DEATHS_PROBABLE',
    'ADA_NEW_DEATHS_CONFIRMED',
    'ADA_NEW_DEATHS_TOTAL',
    'ADA_CUM_DEATHS_SUSPECT',
    'ADA_CUM_DEATHS_PROBABLE',
    'ADA_CUM_DEATHS_CONFIRMED',
    'ADA_CUM_DEATHS_TOTAL',
]

pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

**Step 1:** We normalise Guinea dataset. As you can see from the execution of the code in the following cell, for the Guinea dataset it is safe to use both the new and cumulative totals. However, if we want to average on individual categories, some data is missing. For example, the daily new **confirmed** deaths values exists in only 1 out of 22 days. It is clear that we cannot rely on that category data.

In [ ]:
files = glob.glob(os.path.join(DATA_FOLDER, 'ebola', 'guinea_data', '*.csv'))
columns = ['Date', 'Description', 'Totals']
gdf = pd.concat(pd.read_csv(f, parse_dates=['Date'],
                            usecols=columns,
                            index_col=['Date'])
                for f in files)

# Sort resulting DataFrame by index (Date)
gdf.sort_index(inplace=True)
# Convert Totals column to floats.
gdf['Totals'] = gdf['Totals'].apply(pd.to_numeric, errors='coerce')

# Number of days in Guinea dataset.
number_of_days_in_dataset = len(gdf.index.unique())

# Normalise columns.
conversions = (
    # New cases
    (r"^new.*cases.*suspects$", 'ADA_NEW_CASES_SUSPECT'),
    (r"^new.*cases.*probables$", 'ADA_NEW_CASES_PROBABLE'),
    (r"^new.*cases.*confirmed$", 'ADA_NEW_CASES_CONFIRMED'),
    (r"^total.*new.*cases", 'ADA_NEW_CASES_TOTAL'),
    # Cumulative cases
    (r"^total.*cases.*suspects", 'ADA_CUM_CASES_SUSPECT'),
    (r"^total.*cases.*probables", 'ADA_CUM_CASES_PROBABLE'),
    (r"^total.*cases.*confirmed", 'ADA_CUM_CASES_CONFIRMED'),
    (r"^cumulative.*confirmed.*probable.*suspects", 'ADA_CUM_CASES_TOTAL'),
    # New deaths
    (r"^new.*deaths.*confirmed", 'ADA_NEW_DEATHS_SUSPECT'),
    (r"^new.*deaths.*probables", 'ADA_NEW_DEATHS_PROBABLE'),
    (r"^new.*deaths.*confirmed", 'ADA_NEW_DEATHS_CONFIRMED'),
    (r"^new.*deaths.*registered( today)?$", 'ADA_NEW_DEATHS_TOTAL'),
    # Cumulative deaths
    (r"^total.*deaths.*suspects$", 'ADA_CUM_DEATHS_SUSPECT'),
    (r"^total.*deaths.*probables$", 'ADA_CUM_DEATHS_PROBABLE'),
    (r"^total.*deaths.*confirmed$", 'ADA_CUM_DEATHS_CONFIRMED'),
    (r"^total.*deaths.*confirmed.*probable.*suspects", 'ADA_CUM_DEATHS_TOTAL'),
)

print("Normalising columns for Guinea dataset...", end=' ')
for regex, val in conversions:
    gdf.loc[[re.match(regex, x.lower()) is not None for x in gdf['Description']], 'Description'] = val
print("Done!", end='\n\n')

# Keep only ADA columns.
gdf = gdf.loc[[x.startswith("ADA") for x in gdf['Description']]]

# Make some checks.
print("Checking columns of interest for Guinea dataset...")
print()
print("Checking case-related columns:")
for col in columns_to_check_cases:
    avail = gdf[(gdf.Description == col)].count()['Totals']
    ind = '*' if avail == number_of_days_in_dataset else ' '
    print('[{ind}] {col} ({avail}/{total})'.format(ind=ind, col=col, avail=avail, total=number_of_days_in_dataset))
print()
print("Checking death-related columns:")
for col in columns_to_check_deaths:
    avail = gdf[(gdf.Description == col)].count()['Totals']
    ind = '*' if avail == number_of_days_in_dataset else ' '
    print('[{ind}] {col} ({avail}/{total})'.format(ind=ind, col=col, avail=avail, total=number_of_days_in_dataset))

**Step 2:** We normalise the Liberia dataset. The below code snippet does two jobs:
1. Analyses the data and determines how complete it is, just like with the Guinea data. The output reveals that the Liberia data is way more sparse than the Guinea data, i.e. there is a lot of data missing. For example, there is not indication for daily deaths per individual caregory whatsoever and just half of the samples contain data for the cumulative deaths per individual category. It is clear that we have to average over cumulative totals.

2. Taking into account point 1, tries to fill in some data, so that we can do the analysis. The data might be sparse, but there is something we can exploit. The cumulative data semantics. Since we are averaging per month, even if a lot of "intermediate" cumulative data is missing, we would be very happy if we had the cumulative data for the first and the last day of a month. We can then subtract and divide by the length of the period (days within the month that the period spans). This will give us a good approximation of the average we are after.

Talking more specifically, the code tries to fill in the cumulative cases total, by adding together the individual cumulative values, where they are available. It need to do so only for the first and last day of a given month's data.

In [ ]:
files = glob.glob(os.path.join(DATA_FOLDER, 'ebola', 'liberia_data', '*.csv'))
columns = ['Date', 'Variable', 'National']
ldf = pd.concat(pd.read_csv(f, parse_dates=['Date'],
                            usecols=columns,
                            index_col=['Date'])
                for f in files)

# Sort resulting DataFrame by index (Date)
ldf.sort_index(inplace=True)
# Convert Totals column to floats.
ldf['National'] = ldf['National'].apply(pd.to_numeric, errors='coerce')

# Number of days in Liberia dataset.
number_of_days_in_dataset = len(ldf.index.unique())

# Normalise columns.
conversions = (
    # New cases
    (r"^new.case.s..suspected.$", 'ADA_NEW_CASES_SUSPECT'),
    (r"^new.case.s..probable.$", 'ADA_NEW_CASES_PROBABLE'),
    (r"^^new.case.s..confirmed.$", 'ADA_NEW_CASES_CONFIRMED'),
    # Cumulative cases
    (r"^total.suspected.cases$", 'ADA_CUM_CASES_SUSPECT'),
    (r"^total.probable.cases$", 'ADA_CUM_CASES_PROBABLE'),
    (r"^total.confirmed.cases$", 'ADA_CUM_CASES_CONFIRMED'),
    (r"^cumulative.*confirmed.*probable.*and.*suspected.*cases$", 'ADA_CUM_CASES_TOTAL'),
    # New deaths
    (r"^newly.reported.deaths$$", 'ADA_NEW_DEATHS_TOTAL'),
    # Cumulative deaths
    (r"^total.death.s.in.suspected.cases$", 'ADA_CUM_DEATHS_SUSPECT'),
    (r"^total.death.s.in.probable.cases$", 'ADA_CUM_DEATHS_PROBABLE'),
    (r"^total.death.s.in.confirmed.cases$", 'ADA_CUM_DEATHS_CONFIRMED'),
    (r"^total.death.s.in.confirmed(.|\n)*probable.*suspected.cases$", 'ADA_CUM_DEATHS_TOTAL'),
)

print("Normalising columns for Liberia dataset...", end=' ')
for regex, val in conversions:
    ldf.loc[[re.match(regex, x.lower()) is not None for x in ldf['Variable']], 'Variable'] = val
print("Done!", end='\n\n')

# Keep only ADA columns.
ldf = ldf.loc[[x.startswith("ADA") for x in ldf['Variable']]]

# Make some checks.
print("Checking columns of interest for Liberia dataset...")
print()
print("Checking case-related columns:")
for col in columns_to_check_cases:
    avail = ldf[(ldf.Variable == col)].count()['National']
    ind = '*' if avail == number_of_days_in_dataset else ' '
    print('[{ind}] {col} ({avail}/{total})'.format(ind=ind, col=col, avail=avail, total=number_of_days_in_dataset))
print()
print("Checking death-related columns:")
for col in columns_to_check_deaths:
    avail = ldf[(ldf.Variable == col)].count()['National']
    ind = '*' if avail == number_of_days_in_dataset else ' '
    print('[{ind}] {col} ({avail}/{total})'.format(ind=ind, col=col, avail=avail, total=number_of_days_in_dataset))


# Create a DataFrame to put new rows. Will be later appended to main Liberia DataFrame.
extra_df = pd.DataFrame(data={}, index=[], columns=['Variable', 'National'])

def fill_missing_totals(group):
    """Fill the missing totals for a given group."""
    start_key = group.iloc[[0]].index[0]
    end_key = group.iloc[[-1]].index[0]
    
    # Check if we can fill some TOTAL data for the first and last days of the month...
    for key in [start_key, end_key]:
        try:
            start_cases_total = group.loc[group.Variable == 'ADA_CUM_CASES_TOTAL'].loc[key]['National']
        except KeyError:
            start_cases_total = 0
            try:
                start_cases_suspect = group.loc[group.Variable == 'ADA_CUM_CASES_SUSPECT'].loc[key]['National']
                start_cases_total += start_cases_suspect
            except KeyError: pass
            try:
                start_cases_probable = group.loc[group.Variable == 'ADA_CUM_CASES_PROBABLE'].loc[key]['National']
                start_cases_total += start_cases_probable
            except KeyError: pass
            try:
                start_cases_confirmed = group.loc[group.Variable == 'ADA_CUM_CASES_CONFIRMED'].loc[key]['National']
                start_cases_total += start_cases_confirmed
            except KeyError: pass
            extra_df.loc[key] = ['ADA_CUM_CASES_TOTAL', start_cases_total]
    

def check_monthly_totals(group):
    """Checks whether we can make statistics with monthly totals."""
    totals = [
        'ADA_CUM_CASES_SUSPECT',
        'ADA_CUM_CASES_PROBABLE',
        'ADA_CUM_CASES_CONFIRMED',
        'ADA_CUM_CASES_TOTAL',
        'ADA_CUM_DEATHS_SUSPECT',
        'ADA_CUM_DEATHS_PROBABLE',
        'ADA_CUM_DEATHS_CONFIRMED',
        'ADA_CUM_DEATHS_TOTAL',
    ]
    
    start_key = group.iloc[[0]].index[0]
    end_key = group.iloc[[-1]].index[0]
    print()
    print(group.name, ':', start_key.strftime("%Y-%m-%d"), '-', end_key.strftime("%Y-%m-%d"))
    
    for tot in totals:
        try:
            start_total = group.loc[group.Variable == tot].loc[start_key]['National']
            end_total = group.loc[group.Variable == tot].loc[end_key]['National']
            ind = '*' if end_total > start_total else ' '
        except KeyError as e:
            ind = ' '
        print('[{ind}] {total} ({res})'.format(ind=ind, total=tot, res=end_total-start_total))

print()
print("Data is too sparse, will abandon averaging over daily data.")
print("Checking what we can do with totals...")
ldf.groupby(by=lambda x: x.strftime("%Y-%m")).apply(fill_missing_totals)
# extra_df.index = pd.to_datetime(extra_df.index)
ldf = ldf.append(extra_df).sort_index()
ldf.groupby(by=lambda x: x.strftime("%Y-%m")).apply(check_monthly_totals)


**Step 3:** We normalise the Sierra Leone dataset. We make the following observations:
1. There seem to be enough case data (daily and cumulative). The totals for those are missing, but they can be easily inferred.
2. Regarding the death data, there is no daily data whatsoever, so we cannot use daily values for the aggregation. The good thing is that there are cumulative data, we just need to infer the total.

We follow roughly the same path as with the Liberia dataset. We aggregate as much data as possible for the cumulative totals. We will calculate the average based on the cumulative total difference from the end of a month to the start.

In [ ]:
files = glob.glob(os.path.join(DATA_FOLDER, 'ebola', 'sl_data', '*.csv'))
columns = ['date', 'variable', 'National']
sdf = pd.concat(pd.read_csv(f, parse_dates=['date'],
                            usecols=columns,
                            index_col=['date'])
                for f in files)

# Sort resulting DataFrame by index (Date)
sdf.sort_index(inplace=True)
# Convert Totals column to floats.
sdf['National'] = sdf['National'].apply(pd.to_numeric, errors='coerce')

# Number of days in Sierra Leone dataset.
number_of_days_in_dataset = len(sdf.index.unique())

# Normalise columns.
conversions = (
    # New cases
    (r"^new_suspected$", 'ADA_NEW_CASES_SUSPECT'),
    (r"^new_probable$", 'ADA_NEW_CASES_PROBABLE'),
    (r"^new_confirmed$", 'ADA_NEW_CASES_CONFIRMED'),
    # Cumulative cases
    (r"^cum_suspected$", 'ADA_CUM_CASES_SUSPECT'),
    (r"^cum_probable$", 'ADA_CUM_CASES_PROBABLE'),
    (r"^cum_confirmed$", 'ADA_CUM_CASES_CONFIRMED'),
    # Cumulative deaths
    (r"^death_suspected$", 'ADA_CUM_DEATHS_SUSPECT'),
    (r"^death_probable$", 'ADA_CUM_DEATHS_PROBABLE'),
    (r"^death_confirmed$", 'ADA_CUM_DEATHS_CONFIRMED'),
)

print("Normalising columns for Sierra Leone dataset...", end=' ')
for regex, val in conversions:
    sdf.loc[[re.match(regex, x.lower()) is not None for x in sdf['variable']], 'variable'] = val
print("Done!", end='\n\n')

# Keep only ADA columns.
sdf = sdf.loc[[x.startswith("ADA") for x in sdf['variable']]]

# Make some checks.
print("Checking columns of interest for Sierra Leone dataset...")
print()
print("Checking case-related columns:")
for col in columns_to_check_cases:
    avail = sdf[(sdf.variable == col)].count()['National']
    ind = '*' if avail == number_of_days_in_dataset else ' '
    print('[{ind}] {col} ({avail}/{total})'.format(ind=ind, col=col, avail=avail, total=number_of_days_in_dataset))
print()
print("Checking death-related columns:")
for col in columns_to_check_deaths:
    avail = sdf[(sdf.variable == col)].count()['National']
    ind = '*' if avail == number_of_days_in_dataset else ' '
    print('[{ind}] {col} ({avail}/{total})'.format(ind=ind, col=col, avail=avail, total=number_of_days_in_dataset))

# Create two DataFrame to put new rows. Will be later appended to main Liberia DataFrame.
# Here we need two DataFrames in because we will aggregate data from two different categories.
extra_df = {
    'ADA_CUM_CASES_TOTAL': pd.DataFrame(data={}, index=[], columns=['variable', 'National']),
    'ADA_CUM_DEATHS_TOTAL': pd.DataFrame(data={}, index=[], columns=['variable', 'National']),
}

def fill_missing_totals(group):
    """Fill the missing totals for a given group."""
    start_key = group.iloc[[0]].index[0]
    end_key = group.iloc[[-1]].index[0]
    
    # Check if we can fill some TOTAL data for the first and last days of the month...
    for sus_var, prob_var, conf_var, tot_var in [
        ['ADA_CUM_CASES_SUSPECT', 'ADA_CUM_CASES_PROBABLE', 'ADA_CUM_CASES_CONFIRMED', 'ADA_CUM_CASES_TOTAL'],
        ['ADA_CUM_DEATHS_SUSPECT', 'ADA_CUM_DEATHS_PROBABLE', 'ADA_CUM_DEATHS_CONFIRMED', 'ADA_CUM_DEATHS_TOTAL'],
    ]:
        for key in [start_key, end_key]:
            try:
                start_cases_total = group.loc[group.variable == tot_var].loc[key]['National']
            except KeyError:
                start_cases_total = 0
                try:
                    start_cases_suspect = group.loc[group.variable == sus_var].loc[key]['National']
                    start_cases_total += start_cases_suspect if pd.notnull(start_cases_suspect) else 0.0
                except KeyError: pass
                try:
                    start_cases_probable = group.loc[group.variable == prob_var].loc[key]['National']
                    start_cases_total += start_cases_probable if pd.notnull(start_cases_probable) else 0.0
                except KeyError: pass
                try:
                    start_cases_confirmed = group.loc[group.variable == conf_var].loc[key]['National']
                    start_cases_total += start_cases_confirmed if pd.notnull(start_cases_confirmed) else 0.0
                except KeyError: pass
                extra_df[tot_var].loc[key] = [tot_var, start_cases_total]

def check_monthly_totals(group):
    """Checks whether we can make statistics with monthly totals."""
    totals = [
        'ADA_CUM_CASES_SUSPECT',
        'ADA_CUM_CASES_PROBABLE',
        'ADA_CUM_CASES_CONFIRMED',
        'ADA_CUM_CASES_TOTAL',
        'ADA_CUM_DEATHS_SUSPECT',
        'ADA_CUM_DEATHS_PROBABLE',
        'ADA_CUM_DEATHS_CONFIRMED',
        'ADA_CUM_DEATHS_TOTAL',
    ]
    
    start_key = group.iloc[[0]].index[0]
    end_key = group.iloc[[-1]].index[0]
    print()
    print(group.name, ':', start_key.strftime("%Y-%m-%d"), '-', end_key.strftime("%Y-%m-%d"))
    
    for tot in totals:
        try:
            start_total = group.loc[group.variable == tot].loc[start_key]['National']
            end_total = group.loc[group.variable == tot].loc[end_key]['National']
            ind = '*' if end_total > start_total else ' '
        except KeyError as e:
            print(e)
            ind = ' '
        print('[{ind}] {total} ({res})'.format(ind=ind, total=tot, res=end_total-start_total))

        
# Delete the last day of December, as it makes the cumulative difference negative
# (due to missing data).
sdf.drop(sdf[sdf.index.strftime("%Y-%m-%d") == '2014-12-13'].index, inplace=True)
        
print()
print("Data is too sparse, will abandon averaging over daily data.")
print("Checking what we can do with totals...")
sdf.groupby(by=lambda x: x.strftime("%Y-%m")).apply(fill_missing_totals)

# Merge the two extra DataFrames with the main one.
for _, df in extra_df.items():
    sdf = sdf.append(df).sort_index()

sdf.groupby(by=lambda x: x.strftime("%Y-%m")).apply(check_monthly_totals)

We now have all normalised data in three `DataFrame`s, namely `gdf` for Guinea, `ldf` for Liberia and `sdf` for Sierra Leone. In order to calculate the averages we will `groupby` month and apply the `average_on_totals` function, defined below.

In [ ]:
# Calculate averages.
def average_on_totals_with_daily(group):
    first = group.iloc[:2]  # Get the first available day of the month.
    first.sort_values(first.columns[0], inplace=True)
    last = group.iloc[-2:]  # Get the last available day of the month.
    last.sort_values(last.columns[0], inplace=True)
    
    # Calculate the timespan of the available days (last - first).
    tspan = (last.index.date[0] - first.index.date[0]).days + 1
    
    facc, fday = first.Totals  # fday := first day daily, facc := first day total
    lacc, lday = last.Totals   # lday := last day daily, lacc := last day total
    
    # Calculate average based on timespan.
    average = (lacc - facc) / tspan
    
    # If we only have a day's data available for a month,
    # just return that day's daily count as the average.
    return round(average, 2) if tspan > 1 else round(fday, 2)

def average_on_totals_without_daily(group):
    first = group.iloc[:1]  # Get the first available day of the month.
    last = group.iloc[-1:]  # Get the last available day of the month.
    
    # Calculate the timespan of the available days (last - first).
    tspan = (last.index.date[0] - first.index.date[0]).days + 1
    
    facc = first.Totals[0]
    lacc = last.Totals[0]
    
    average = (lacc - facc) / tspan
    
    return round(average, 2) if tspan > 1 else np.NaN
    

# Calculate averages for Guinea
print("---> Making calculations for Guinea")
# Define criteria
cases_criteria = [x in ['ADA_CUM_CASES_TOTAL', 'ADA_NEW_CASES_TOTAL'] for x in gdf.Description]
deaths_criteria = [x in ['ADA_CUM_DEATHS_TOTAL', 'ADA_NEW_DEATHS_TOTAL'] for x in gdf.Description]

print("-> Calculating average cases per day for Guinea")
criteria = cases_criteria
guinea_cdf = gdf.loc[criteria] \
                 .groupby(by=lambda x: x.strftime("%Y-%m")) \
                 .apply(average_on_totals_with_daily)
guinea_cdf.name = 'Cases'
print(guinea_cdf)

print("-> Calculating average deaths per day for Guinea")
criteria = deaths_criteria
guinea_ddf = gdf.loc[criteria] \
                 .groupby(by=lambda x: x.strftime("%Y-%m")) \
                 .apply(average_on_totals_with_daily)
guinea_ddf.name = 'Deaths'
print(guinea_ddf)

print()

# Calculate averages for Liberia
print("---> Making calculations for Liberia")
# Define criteria
cases_criteria = [x in ['ADA_CUM_CASES_TOTAL', 'ADA_NEW_CASES_TOTAL'] for x in ldf.Variable]
deaths_criteria = [x in ['ADA_CUM_DEATHS_TOTAL', 'ADA_NEW_DEATHS_TOTAL'] for x in ldf.Variable]

ldf.columns = ['Totals' if x == 'National' else x for x in ldf.columns]

print("-> Calculating average cases per day for Liberia")
criteria = cases_criteria
liberia_cdf = ldf.loc[criteria] \
                 .groupby(by=lambda x: x.strftime("%Y-%m")) \
                 .apply(average_on_totals_without_daily)
liberia_cdf.name = 'Cases'
print(liberia_cdf)

print("-> Calculating average deaths per day for Liberia")
criteria = deaths_criteria
liberia_ddf = ldf.loc[criteria] \
                 .groupby(by=lambda x: x.strftime("%Y-%m")) \
                 .apply(average_on_totals_with_daily)
liberia_ddf.name = 'Deaths'
print(liberia_ddf)

print()

# Calculate averages for Sierra Leone
print("---> Making calculations for Sierra Leone")
# Define criteria
cases_criteria = [x in ['ADA_CUM_CASES_TOTAL', 'ADA_NEW_CASES_TOTAL'] for x in sdf.variable]
deaths_criteria = [x in ['ADA_CUM_DEATHS_TOTAL', 'ADA_NEW_DEATHS_TOTAL'] for x in sdf.variable]

sdf.columns = ['Totals' if x == 'National' else x for x in sdf.columns]

print("-> Calculating average cases per day for Sierra Leone")
criteria = cases_criteria
sierra_cdf = sdf.loc[criteria] \
                .groupby(by=lambda x: x.strftime("%Y-%m")) \
                .apply(average_on_totals_without_daily)
sierra_cdf.name = 'Cases'
print(sierra_cdf)

print("-> Calculating average deaths per day for Sierra Leone")
criteria = deaths_criteria
sierra_ddf = sdf.loc[criteria] \
                .groupby(by=lambda x: x.strftime("%Y-%m")) \
                .apply(average_on_totals_without_daily)
sierra_ddf.name = 'Deaths'
print(sierra_ddf)

for df in [guinea_cdf, guinea_ddf, liberia_cdf, liberia_ddf, sierra_cdf, sierra_ddf]:
    df.index.name = 'Month'

Now, we just need to combine the resulting `DataFrames` containing the averages to one singe `DataFrame`. We chose to indicate missing values with `N/A`s rather than with zeroes, in order to be clear that the data is missing and not mistakenly considered zero.

In [ ]:
months = ['2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12']
variables = ['Cases', 'Deaths']
countries = ['Guinea', 'Liberia', 'Sierra Leone']
index = pd.MultiIndex.from_product((months, variables), names=('Month', 'Variable'))
DF = pd.DataFrame(data='N/A',
            index=index,
            columns=countries)

# Assign cases
for df, country in [(guinea_cdf, 'Guinea'), (liberia_cdf, 'Liberia'), (sierra_cdf, 'Sierra Leone')]:
    for month in df.index:
        DF.loc[month, 'Cases'][country] = df[month]

# Assign deaths
for df, country in [(guinea_ddf, 'Guinea'), (liberia_ddf, 'Liberia'), (sierra_ddf, 'Sierra Leone')]:
    for month in df.index:
        DF.loc[month, 'Deaths'][country] = df[month]

DF


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

#### Solution

First, we read our 9 excel files and store them into 9 different DataFrames:

In [ ]:
mid1 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID1.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)
mid2 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID2.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)
mid3 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID3.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)
mid4 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID4.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)
mid5 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID5.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)
mid6 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID6.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)
mid7 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID7.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)
mid8 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID8.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)
mid9 = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'MID9.xls'), sheetname = 'Sheet 1', index_col = 0, header = None)

We give names to the index and the column:

In [ ]:
mid1.index.name = mid2.index.name = mid3.index.name = mid4.index.name = 'Taxonomy'
mid5.index.name = mid6.index.name = mid7.index.name = mid8.index.name = mid9.index.name = 'Taxonomy'

mid1.columns = mid2.columns = mid3.columns = mid4.columns = ['Count']
mid5.columns = mid6.columns = mid7.columns = mid8.columns = mid9.columns = ['Count']

We combine the 9 DataFrames into a single DataFrame, on axis 0 (rows):

In [ ]:
mid = pd.concat([mid1, mid2, mid3, mid4, mid5, mid6, mid7, mid8, mid9], axis = 0, 
                keys=['MID1', 'MID2', 'MID3', 'MID4', 'MID5', 'MID6', 'MID7', 'MID8', 'MID9'], names = ["Barcode"])

mid

We read the metadata:

In [ ]:
meta = pd.read_excel(os.path.join(DATA_FOLDER, 'microbiome', 'metadata.xls'), sheetname = 'Sheet1', index_col = 0, header = 0)
meta

We rename the columns using lower-case letters, except for the first letter, just so that they look prettier:

In [ ]:
meta.index.name = "Barcode"
meta.rename(columns = {"GROUP": "Group", "SAMPLE": "Sample"}, inplace = True)
meta

We merge the two DataFrames, mid and meta, into a single merged DataFrame and we fill all the NaN values with the tag "unknown". We also remove the index, so that we can reset it from scratch, after we decide exactly how we want to present our data:

In [ ]:
merged = pd.merge(mid, meta, how='outer', left_index=True, right_index=True)
# merged.index # True

merged = merged.fillna("unknown")

merged = merged.reset_index()

merged

We create a pivot table from the merged data, where the taxonomy is the index, the group, sample and barcode are the columns and the count is the value. We fill each blank cell with the value 0. (A blank cell does not mean that the value is not known, but rather that a particular microorganism was not encountered in a specific group of extractions, a specific barcode and with a specific sample type).

We decided to use this type of representation for the data, so as to allow the reader to observe, in a single row, how much a specific microorganism has been detected in various extractions. Since a group can contain several samples and barcodes, we considered appropriate to set it as our first level of columns. Each group (apart from the one called "EXTRACTION CONTROL") seems to use both stool and tissue samples. Thus, we used the sample as our second level of columns. Finally, our last level of columns is the barcode, which seems to be simply an id, without carrying any significant information about the extractions.

In [ ]:
final = merged.pivot_table(index=['Taxonomy'], columns=['Group', 'Sample', 'Barcode'], fill_value = 0,
                           aggfunc=np.sum, margins=True)
final.head()

We rename the columns using lower-case letters, except for the first letter, just so that they look prettier:

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
df = pd.read_excel(DATA_FOLDER+'/titanic.xls', header=0)
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# find which columns contain at least one NaN
df.columns[pd.isnull(df).sum() > 0].tolist()

#  COLUMN: pclass

In [ ]:
df.pclass.value_counts()

There's no need to analyze more the column pclass.

#  COLUMN: survived

In [ ]:
df.survived.value_counts()

There's no need to analyze more the column survived.

#  COLUMN: name

I do not think that there should be any analysis on the column name, regarding the range of values

#  COLUMN: sex 

In [ ]:
df.sex.value_counts()

The values of sex column shall be treated as categorical data

In [ ]:
sex_map = {'male': 0, 'female': 1}
df.sex.replace(sex_map, inplace=True)

#  COLUMN: age

In [ ]:
df[df.age.isnull()]

In [ ]:
age = df.age.dropna()
age.describe()

The value range of the age column goes from newborn babies(0.166) to 80 years.

# COLUMN: sibsp

sibsp col is of type int64 and the range of values: [0 - 8]

In [ ]:
df.sibsp.value_counts()

# COLUMN: parch

parch is of type int64 and the range of values: [0 - 9]

In [ ]:
df.parch.value_counts()

# COLUMN: ticket

Thic column is of type object and I do not think there should not be any analysis regarding the range values.

# COLUMN: fare

This column is of type float64 and there are Nan values

In [ ]:
fare = df.fare.dropna()
fare.describe()

There are some passengers that their fare is 0

# COLUMN: cabin

cabin column is of type object

In [ ]:
cabin = df.cabin.dropna()
cabin.describe()

# COLUMN: embarked

In [ ]:
emb = df.embarked.dropna()
emb.value_counts()

Passengers where embarked from three different ports
In this case we shall use categorical data and represent {S = 0, C = 1, Q = 2}

In [ ]:
embarked_map = {'S': 0, 'C': 1, 'Q' : 2}
df.embarked.replace(embarked_map, inplace=True)

In [ ]:
df.embarked.value_counts()

Regarding the columns boat, body, home.dest there is not any range for the values that makes sense.
Also, the data in the three columns vary, in a way that if we categorize them, it would not improve our analysis.

# QUESTION 2

Plot histograms for the travel class, embarkation port, sex and age attributes. For the latter one, use discrete decade intervals. 

# Histogram for the travel class

In [ ]:
df.hist(column='pclass')

# Histogram for the embarkation port

where 0 stands for port 'S' (Southampton), 1 for 'C' (Cherbourg) and 2 for 'Q' (Queenstown) 

In [ ]:
df.hist('embarked')

# Histogram for the sex

where 0 stands for male, 1 for female

In [ ]:
df.hist(column='sex')

# Histogram for the age

In [ ]:
df.hist(column='age')

# Histogram for age using discrete decade intervals 

In [ ]:
bin_range = np.arange(0, 100+10, 10)
out, bins = pd.cut(df.age, bins = bin_range, include_lowest=True, right=False, retbins=True)
out.value_counts(sort=False).plot.bar()

# QUESTION 3

Calculate the proportion of passengers by cabin floor. Present your results in a pie chart.

In [ ]:
df.cabin.isnull().any()

In [ ]:
df_grouped = df[['pclass','cabin']].dropna(axis=0).groupby(by = df.cabin)

In [ ]:
df_grouped.count()

In order to group by floor we add to the dataframe a column that descrives the floor.
This information is obtained by the first letter of the attribute cabin.

In [ ]:
df_grouped2 = df[['pclass','cabin']]

In [ ]:
df_grouped2 = df_grouped2.dropna(axis=0)
df_grouped2.head()

In [ ]:
def cabin_floor(cabin):
    return cabin[:1]

In [ ]:
df_grouped2['cabin_floor'] = df_grouped2['cabin'].apply(lambda x: cabin_floor(x))

In [ ]:
df_grouped2.head()

In [ ]:
df_grouped2 = df_grouped2.drop('cabin', axis=1).groupby(by='cabin_floor')

So now we present the proportion of passengers by cabin floor

In [ ]:
df_c = df_grouped2.count()
df_c

In [ ]:
labels = df_c.index
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.subplot(aspect=1)
plt.rcParams["figure.figsize"] = fig_size
plt.pie(df_c['pclass'], labels=labels,  autopct='%1.2f%%', shadow=True, radius=1.5)

# QUESTION 4

For each travel class, calculate the proportion of the passengers that survived. Present your results in pie charts.

In [ ]:
cl_gr = df[['survived','pclass','name']].groupby(by=['pclass','survived'])

In [ ]:
cl_gr.count()

In [ ]:
df_survived = df[['pclass','survived']].groupby('pclass')['survived'].sum()
df_survived

In [ ]:
df_class_grouped = df[['pclass','survived']].groupby('pclass')['survived'].count()
df_class_grouped

In [ ]:
df_proportion_survived = df_survived/df_class_grouped * 100
df_proportion_survived

In [ ]:
type(df_proportion_survived)

In [ ]:
df_proportion_survived = df_proportion_survived.to_frame()

In [ ]:
type(df_proportion_survived)

In [ ]:
df_proportion_survived['not survived'] = df_proportion_survived['survived'].apply(lambda x: 100 - x)
df_proportion_survived

In [ ]:
labels = ['survived', 'not survived']
prop = ['proportion']
df_temp = pd.DataFrame(index=labels, columns=prop)
#for i, row in df_proportion_survived.iteritems():
#    df_temp.loc['survived'] = df_proportion_survived[i]
#    df_temp.loc['not survived'] = 100 - df_proportion_survived[i]
#    plt.subplot(aspect=1)
#    plt.pie(df_temp, labels=labels,  autopct='%1.2f%%', shadow=True, radius=1.5)
    
    
    
fig, axes = plt.subplots(1, 3, figsize=(14, 8))
for i, (idx, row) in enumerate(df_proportion_survived.iterrows()):
    ax = axes[i % 3]
    row = row[row.gt(row.sum() * .01)]
    ax.pie(row, labels=row.index, startangle=0)
    title = 'pclass '+ str(idx)
    ax.set_title(title)
fig.subplots_adjust(wspace=.4)

# QUESTION 5

Calculate the proportion of the passengers that survived by travel class and sex. Present your results in a single histogram.

In [ ]:
df[df['survived'] == 1].hist(column= 'survived', by=['sex', 'pclass'])
#(0,1) stands for male in class 1, (0,2) for male in class 2, (1,1) for female in class 1 etc 

In [ ]:
grouped_class_sex = df.groupby(by=['pclass', 'sex'])['survived']
df_class_sex = grouped_class_sex.sum()

In [ ]:
df_class_sex = df_class_sex.to_frame()

In [ ]:
df_class_sex.plot(kind='bar')

In [ ]:
df_class_sex

In [ ]:
survived_class_and_sex = df[df.survived == 1].pivot_table(index = 'pclass', columns = 'sex', values='survived', aggfunc='sum')
survived_class_and_sex['Total'] = survived_class_and_sex.sum(axis=1)
percentage = survived_class_and_sex.loc[:,[0,1]].div(survived_class_and_sex['Total'], axis=0) * 100
percentage

In [ ]:
percentage.hist()

The underline graph presents the result in a better way

In [ ]:
percentage.plot(kind='bar')

# QUESTION 6

Create 2 equally populated age categories and calculate survival proportions by age category, travel class and sex. Present your results in a DataFrame with unique index.

We consider that age values < 1, are babies. Since their name is provided we think that removing these values from the dataframe would not be right.

In [ ]:
df.dropna(subset=['age'], inplace=True)
df.sort_values(by='age', ascending=True, inplace=True)
df.shape

In [ ]:
df_age = df['age']
df_age = df_age.sort_values(ascending=True)

In [ ]:
populated = np.split(df_age,2)
len(populated[0])

In [ ]:
populated[0].tail()

In [ ]:
populated[1].head()

We created 2 equally age categories, they are separated on the age 28. 

In [ ]:
df_cat1 = df.loc[:523, :]
df_cat1.shape

In [ ]:
labels = pd.cut(df.age, bins=2, labels=['age_category1','age_category2'])
df['age_category'] = labels
last_df = df[df.survived == 1].pivot_table(index = ['pclass','sex','age_category'], values='survived', aggfunc='sum')
last_df

In [ ]:
last_df.sum(axis=0)

In [ ]:
percentage = last_df.div(last_df.sum(axis=0)) * 100
percentage